# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 19 2022 4:41PM,237700,18,HH6053.2,Hush Hush,Released
1,May 19 2022 4:30PM,237698,19,ACG-2102490121,ACG North America LLC,Released
2,May 19 2022 4:30PM,237698,19,ACG-2102490124,ACG North America LLC,Released
3,May 19 2022 4:30PM,237698,19,ACG-2102490125,ACG North America LLC,Released
4,May 19 2022 4:30PM,237698,19,ACG-2102490126,ACG North America LLC,Released
5,May 19 2022 4:30PM,237698,19,ACG-2102490127,ACG North America LLC,Released
6,May 19 2022 4:17PM,237697,10,SO202201394,Yusen – 3D Matrix,Executing
7,May 19 2022 4:17PM,237696,10,8041612,"Herbivore Botanicals, LLC.",Released
8,May 19 2022 4:17PM,237696,10,8041636,"Herbivore Botanicals, LLC.",Released
9,May 19 2022 4:17PM,237696,10,8041640,"Herbivore Botanicals, LLC.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,237694,Released,1
25,237696,Released,3
26,237697,Executing,1
27,237698,Released,5
28,237700,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237694,NaN,NaN,1.0
237696,NaN,NaN,3.0
237697,NaN,1.0,NaN
237698,NaN,NaN,5.0
237700,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3.0,0.0,2.0
237597,0.0,0.0,1.0
237598,0.0,0.0,1.0
237602,0.0,0.0,1.0
237608,3.0,19.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237560,3,0,2
237597,0,0,1
237598,0,0,1
237602,0,0,1
237608,3,19,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,0,2
1,237597,0,0,1
2,237598,0,0,1
3,237602,0,0,1
4,237608,3,19,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237560,3,,2
1,237597,,,1
2,237598,,,1
3,237602,,,1
4,237608,3,19,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 19 2022 4:41PM,237700,18,Hush Hush
1,May 19 2022 4:30PM,237698,19,ACG North America LLC
6,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix
7,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC."
10,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions
11,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions
12,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC"
13,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc."
14,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC"
15,May 19 2022 2:56PM,237683,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 19 2022 4:41PM,237700,18,Hush Hush,,,1
1,May 19 2022 4:30PM,237698,19,ACG North America LLC,,,5
2,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,,1,
3,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",,,3
4,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,,,1
5,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,,,1
6,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",,,1
7,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",,,1
8,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC",,,1
9,May 19 2022 2:56PM,237683,10,ISDIN Corporation,,,47


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:41PM,237700,18,Hush Hush,1,,
1,May 19 2022 4:30PM,237698,19,ACG North America LLC,5,,
2,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,,1,
3,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3,,
4,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1,,
5,May 19 2022 4:00PM,237693,10,Acute Outpatient Solutions,1,,
6,May 19 2022 3:35PM,237690,19,"Emersa Waterbox, LLC",1,,
7,May 19 2022 3:34PM,237689,16,"Goodwin Biotechnology, Inc.",1,,
8,May 19 2022 3:29PM,237688,19,"Emersa Waterbox, LLC",1,,
9,May 19 2022 2:56PM,237683,10,ISDIN Corporation,47,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:41PM,237700,18,Hush Hush,1,,
1,May 19 2022 4:30PM,237698,19,ACG North America LLC,5,,
2,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,,1,
3,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3,,
4,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:41PM,237700,18,Hush Hush,1.0,NaN,NaN
1,May 19 2022 4:30PM,237698,19,ACG North America LLC,5.0,NaN,NaN
2,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,NaN,1.0,NaN
3,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3.0,NaN,NaN
4,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 19 2022 4:41PM,237700,18,Hush Hush,1.0,0.0,0.0
1,May 19 2022 4:30PM,237698,19,ACG North America LLC,5.0,0.0,0.0
2,May 19 2022 4:17PM,237697,10,Yusen – 3D Matrix,0.0,1.0,0.0
3,May 19 2022 4:17PM,237696,10,"Herbivore Botanicals, LLC.",3.0,0.0,0.0
4,May 19 2022 4:07PM,237694,10,Acute Outpatient Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2138994,71.0,29.0,3.0
16,712952,3.0,0.0,0.0
17,237654,0.0,1.0,0.0
18,950538,4.0,0.0,0.0
19,713076,7.0,0.0,0.0
20,712874,22.0,0.0,3.0
22,475281,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2138994,71.0,29.0,3.0
1,16,712952,3.0,0.0,0.0
2,17,237654,0.0,1.0,0.0
3,18,950538,4.0,0.0,0.0
4,19,713076,7.0,0.0,0.0
5,20,712874,22.0,0.0,3.0
6,22,475281,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,71.0,29.0,3.0
1,16,3.0,0.0,0.0
2,17,0.0,1.0,0.0
3,18,4.0,0.0,0.0
4,19,7.0,0.0,0.0
5,20,22.0,0.0,3.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,16,Released,3.0
2,17,Released,0.0
3,18,Released,4.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,18,19,20,22
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,29.0,0.0,1.0,0.0,0.0,0.0,0.0
Released,71.0,3.0,0.0,4.0,7.0,22.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,29.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,71.0,3.0,0.0,4.0,7.0,22.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,29.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,71.0,3.0,0.0,4.0,7.0,22.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()